# Transfer Learning with Tensorflow Part 1: Feature Extraction

 Transfer learnin is leveraging a working model's existing architecture and learned patterns
 for our own problem.

 2 benefits
 1. can leverage existing neural network architecture proven to work on to our own.
 2. can leverage a working neural network architecute which has already learned patterns on
 similalr data to our own, then we can adapt those patterns to our own data.
  

In [ ]:
# Are we using a gpu?
!nvidia-smi


# Downloading and becoming one with the data

In [ ]:
#Get data(10% of 10 food classes from food101)

import zipfile

#Download the data
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip

In [ ]:
#unzip the downladed file
zip_ref = zipfile.ZipFile("10_food_classes_10_percent.zip","r")
zip_ref.extractall()
zip_ref.close()

In [ ]:
#How many images in each folder?

import os

# Walk thr0ugh 10 percent data directory and list nuber of files
for dirpath, dirnames, filenames in os.walk("10_food_classes_10_percent"):
  print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'.")

## Creating data loaders(preparing the data)
we'll use the `ImageDataGenerator` class to load in our images in batches.

In [ ]:
#Setup data inputs
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Hyperparameters
IMAGE_SHAPE = (224,224)
BATCH_SIZE = 32


train_dir = "10_food_classes_10_percent/train/"
test_dir = "10_food_classes_10_percent/test/"

train_datagen = ImageDataGenerator(rescale = 1/255.)
test_datagen = ImageDataGenerator(rescale = 1/255.)

In [ ]:
print("Training images:")
train_data_10_percent = train_datagen.flow_from_directory(train_dir,
                                                          target_size = IMAGE_SHAPE,
                                                          batch_size = BATCH_SIZE,
                                                          class_mode = "categorical")
print("testing images:")
test_data_10_percent = test_datagen.flow_from_directory(test_dir,
                                                          target_size = IMAGE_SHAPE,
                                                          batch_size = BATCH_SIZE,
                                                          class_mode = "categorical")


## Setting up callbacks (things to run whilst our model trains)

Callbacks are extra functiionality you can add to your models to be performed during or after training. Some of the most popular callbacks:

* Tracking experiments with the TensorBoard callback

* Model checkpoint with the ModelCheckpoint callback

* Stopping a model from training (before it training too long and overfits) with the EarlyStopping callback

In [ ]:
import tensorflow as tf

In [ ]:
#Create TensorBoard callback (funtionized because we need to creae a new one for each model)
import datetime

def create_tensorboard_callback(dir_name, experiment_name):
  log_dir = dir_name + "/" + experiment_name + "/" + datetime.datetime.now().strftime("%y%m%d-%H%M%S")
  tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir = log_dir)
  print(f"Saving TensorBoard log files to: {log_dir}")
  return tensorboard_callback

In [ ]:
import tensorflow as tf
import datetime
import os

def create_tensorboard_callback(dir_name, experiment_name):
    """Creates a TensorBoard callback that logs training metrics to a directory."""

    # Ensure directory exists
    log_dir = os.path.join(dir_name, experiment_name, datetime.datetime.now().strftime("%y%m%d-%H%M%S"))
    os.makedirs(log_dir, exist_ok=True)  # Create directory if it doesn't exist

    # Create TensorBoard callback
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)

    print(f" Saving TensorBoard logs to: {log_dir}")
    return tensorboard_callback


Note: You can customize the directory where your TensorBoard logs (model training metrics )
saved to whatever you like. The log_dir parameter we've created above is only one option.

## Creating models using TensorFlow Hub

In the past we've used TensorFlow to create our own models layer by layer from scratch.

Now we're going to do similar process, except the majority are used from teh TensorFlow HUb.

we access models using : https://tfhub.dev/

This the feature vector model that we are using. https://www.kaggle.com/models/tensorflow/efficientnet/TensorFlow2/b0-classification/1

In [ ]:
resnet_url = "https://tfhub.dev/google/imagenet/resnet_v2_50/feature_vector/5"
efficientnet_url = "https://tfhub.dev/tensorflow/efficientnet/b0/feature-vector/1"


In [ ]:
!pip install tensorflow_hub --upgrade  #Upgrade TensorFlow Hub to the latest version
!pip install --upgrade tensorflow     #Upgrade Tensorflow to the latest version

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers

# Create a custom wrapper class for the hub.KerasLayer
class HubLayerWrapper(layers.Layer):
    def __init__(self, model_url, **kwargs):
        super(HubLayerWrapper, self).__init__(**kwargs)
        self.model_url = model_url

    def build(self, input_shape):
        self.hub_layer = hub.KerasLayer(self.model_url, trainable=False)
        self.hub_layer.build(input_shape)
        super(HubLayerWrapper, self).build(input_shape)

    def call(self, inputs):
        return self.hub_layer(inputs)

def create_model(model_url, num_classes=10):
    """
    Create a model with TensorFlow Hub's feature extraction layer and a custom classification head.

    Args:
        model_url (str): A TensorFlow Hub feature extraction URL.
        num_classes (int): Number of output neurons in the output layer.

    Returns:
        A Keras model.
    """


    # Input layer
    inputs = layers.Input(shape=(224, 224, 3))

    # Feature extraction using the custom HubLayerWrapper
    feature_extractor = HubLayerWrapper(model_url)(inputs)

    # Classification layer
    outputs = layers.Dense(num_classes, activation='softmax', name="output_layer")(feature_extractor)

    # Create and return the model
    model = tf.keras.Model(inputs, outputs)

    return model


In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers

# Optimized Hub Layer Wrapper
class HubLayerWrapper(layers.Layer):
    def __init__(self, model_url, trainable=False, **kwargs):
        super(HubLayerWrapper, self).__init__(**kwargs)
        self.hub_layer = hub.KerasLayer(model_url, trainable=trainable)  # Initialize once

    def call(self, inputs):
        return self.hub_layer(inputs)  # Directly call the hub layer


In [ ]:
def create_model(model_url, num_classes=10):
    """
    Create a model with TensorFlow Hub's feature extraction layer and a classification head.

    Args:
        model_url (str): A TensorFlow Hub feature extraction URL.
        num_classes (int): Number of output neurons in the output layer.

    Returns:
        A Keras model.
    """

    # Input layer
    inputs = layers.Input(shape=(224, 224, 3))

    # Feature extraction using the optimized wrapper
    feature_extractor = HubLayerWrapper(model_url, trainable=False)(inputs)

    # Classification layer
    outputs = layers.Dense(num_classes, activation='softmax', name="output_layer")(feature_extractor)

    # Create and return the model
    model = tf.keras.Model(inputs, outputs)

    return model


In [ ]:
"""import tensorflow as tf
import tensorflow_hub as hub

# Define input
inputs = tf.keras.Input(shape=(224, 224, 3), dtype=tf.float32)

# Load TensorFlow Hub Layer
hub_layer = hub.KerasLayer("https://tfhub.dev/google/imagenet/resnet_v2_50/classification/5", trainable=False)

print(hub_layer.get_config())"""

In [ ]:
"""dummy_input = tf.ones((1, 224, 224, 3))  # Batch size 1
output = hub_layer(dummy_input)
print("Output shape:", output.shape)"""

In [ ]:
"""inputs = tf.keras.Input(shape=(224, 224, 3), dtype=tf.float32)"""

In [ ]:
"""print(inputs.shape)    """

In [ ]:
"""import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers

# Define a wrapper for the TensorFlow Hub layer
class HubLayerWrapper(tf.keras.layers.Layer):
    def __init__(self, model_url, **kwargs):
        super(HubLayerWrapper, self).__init__(**kwargs)
        self.model_url = model_url
        self.feature_extractor_layer = hub.KerasLayer(model_url, trainable=False, name="feature_extraction_layer")

    def build(self, input_shape):
        self.feature_extractor_layer.build(input_shape)

    def call(self, inputs):
        return self.feature_extractor_layer(inputs)

# Define the function to create the model
def create_model(model_url, num_classes=10):

    #Creates a Keras Sequential model using a TensorFlow Hub feature extractor.

    #Args:
    #model_url (str): URL of the TensorFlow Hub model.
    #num_classes (int): Number of classes for classification.

    #Returns:
    #tf.keras.Sequential: A compiled Keras model.

    feature_extractor_layer = HubLayerWrapper(model_url, input_shape=(224, 224, 3))

    model = tf.keras.Sequential([
        feature_extractor_layer,
        layers.Dense(num_classes, activation='softmax', name="output_layer")
    ])

    return model

# Example usage
model_url = "https://tfhub.dev/google/imagenet/resnet_v2_50/feature_vector/5"
resnet_model = create_model(model_url, num_classes=10)

# Print model summary
#resnet_model.build((None, 224, 224, 3))  # Ensure correct input shape
resnet_model.summary()
"""

#gives the same function as above


# Creating and testing ResNet TensorFlow Hub Feature Extraction model

In [ ]:

# Define the URL for ResNet or any other model you want to use from TensorFlow Hub
resnet_url = "https://tfhub.dev/google/imagenet/resnet_v2_50/feature_vector/5"

# Create the model
resnet_model = create_model(resnet_url, train_data_10_percent.num_classes)

# Print the model summary
resnet_model.summary()


In [ ]:
#compile our resnet model
resnet_model.compile(loss = "categorical_crossentropy",
                     optimizer = tf.keras.optimizers.Adam(),
                     metrics = ["accuracy"])

In [ ]:
def convert_to_tf_dataset(directory_iterator, batch_size=32):
    return tf.data.Dataset.from_generator(
        lambda: directory_iterator,
        output_signature=(
            tf.TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32),
            tf.TensorSpec(shape=(None,), dtype=tf.int32)
        )
    ).batch(batch_size).prefetch(tf.data.AUTOTUNE)

# Convert train and test datasets
train_data_10_percent = convert_to_tf_dataset(train_data_10_percent)
test_data_10_percent = convert_to_tf_dataset(test_data_10_percent)

# Resize images
def resize_images(batch_images, label):
    # Ensure batch_images is 4D (batch_size, height, width, channels)
    print(batch_images.shape)  # Check if the shape is (batch_size, height, width, channels)
    resized_images = tf.image.resize(batch_images, (224, 224))
    print(resized_images.shape)  # After resize (batch_size, 224, 224, 3)
    return resized_images, label

# Apply the map function for resizing images
train_data_10_percent = train_data_10_percent.map(resize_images, num_parallel_calls=tf.data.AUTOTUNE)
test_data_10_percent = test_data_10_percent.map(resize_images, num_parallel_calls=tf.data.AUTOTUNE)


In [ ]:
#fit the resnet model to the data
resnet_history = resnet_model.fit(train_data_10_percent,
                                  epochs =5,
                                  steps_per_epoch = len(train_data_10_percent)-1,
                                  validation_data = test_data_10_percent,
                                  validation_steps= len(test_data_10_percent)-1,
                                  callbacks = [create_tensorboard_callback(dir_name ="tensorflow_hub",
                                                                           experiment_name="resnet50v1")])


In [ ]:
#funtion to plot loss curves..
import matplotlib.pyplot as plt

#PLT THE VALIDATION AND TRAINING CURVES

def plot_loss_curves(history):
  """
  returns separate loss curves for training and validation metrics.

  Args:
  history: TensorFlow History object.

  Returns:
  Plots of training/validation loss and accuracy metrics.
  """
  loss = history.history["loss"]
  val_loss = history.history["val_loss"]

  accuracy = history.history["accuracty"]
  val_accuracy = history.history["val_accuracy"]

  epochs = range(len(history.history["loss"]))

  # Plot loss
  plt.plot(epochs, loss, label= "training_loss")
  plt.plot(epochs, val_loss, label = "val_loss")
  plt.title("Loss")
  plt.xlabel("Epochs")
  plt.legend()

  # Plot accuracy
  plt.figure()
  plt.plot(epochs, accuracy, label = "training_accuracy")
  plt.plot(epochs, val_accuracy, label = "val_accuracy")
  plt.title("Accuracy")
  plt.xlabel("Epochs")
  plt.legend();

In [ ]:
#function call
plot_loss_curves(resnet_history)

## Creating and testing EfficientNetB0 TensorFlow Hub Feature Extraction model.


In [ ]:
#Create EfficientNetB0 feature extractor model
efficient_model = create_model(efficientnet_url, num_classes = train_data_10_percent.num_classes)

#Compile the model
efficient_model.compile(loss = "categorical_crossentropy",
                        optimizer = tf.keras.optimizers.Adam(),
                        metrics = ["accuracy"])


In [ ]:
#Fit EfficientNet model to the training data
efficient_history = efficient_model.fit(train_data_10_percent,
                                         epochs = 5,
                                         steps_per_epoch = len(train_data_10_percent),
                                         validation_data = test_data_10_percent,
                                         validation_steps = len(test_data_10_percent),
                                         callbacks = [create_tensorboard_callback(dir_name = "tensorflow_hub",
                                                                                   experiment_name = "efficientnetb0v1")])

In [ ]:
#how many layers does our efficientnetb0 feature extractor have
len(efficientnet_model.layer[0].weights)

In [ ]:
#plotting loss curves
plot_loss_curves(efficient_history)

## Different types of transfer learning

* transfer learning - using an existing model with no changes
* Feature extraction transfer learning - use the prelearned patterns of existing model and adjust output layer for our own output.
* Fine-tuning transfer learning - use the prelearned patters of existing model and "fine-tune" many or all of the underlying layers(including output layer)

# Comparing our models results using TensorBoard

In [ ]:
#upload TensorBoard dev records this will upload to tensorboard. (it is shutdown so no use)
!tensorboard dev upload --logdir ./tensorflow_hub/ \
--name "EfficientNetB0 vs. ResNet50V2" \
--description "Compareing two different TF Hub feature extraction model architecture using 10% training data" \
--one_shot

In [ ]:
!tensorboard dev list

In [ ]:
#to delete
#!tensorboard dev delete --experiment_id (you specify id here without brackets)